# Final Project of Introduction to Bioinformatics

## Comparative Mitochondrial Genomics and Phylogenetic Analysis



### Exploration and Reflection

As we proceed with our analysis of mitochondrial DNA for phylogenetic tree construction, it is valuable to contemplate a few questions. These inquiries aim to facilitate a more thorough understanding of the roles and characteristics of mitochondrial DNA in the context of evolutionary biology:

1. **Maternal Inheritance and Its Implications**: How does the maternal inheritance of mitochondrial DNA simplify our understanding of evolutionary lineage compared to nuclear DNA, which undergoes recombination? What unique insights can this aspect provide in tracing the evolutionary history of species?

2. **Mutation Rate and Evolutionary Insights**: Mitochondrial DNA mutates at a faster rate than nuclear DNA. How does this characteristic make mtDNA a more sensitive tool for detecting recent evolutionary events and relationships among closely related species? Can you think of any specific scenarios or studies where this property of mtDNA has been particularly instrumental?

Reflect on these questions as you work through the project, and consider how the properties of mitochondrial DNA enhance its value and applicability in evolutionary biology and beyond. Provide your answer either in this notebook, or in your report (if you had one).

<blockquote style="font-family:Arial; color:red; font-size:16px; border-left:0px solid red; padding: 10px;">
    <strong>Don't forget to answer these questions!</strong>
</blockquote>

### Step 0: Installing Necessary Packages

In [2]:
import sys
import subprocess
import pkg_resources

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

REQUIRED_PACKAGES = [
    'biopython',
    'pandas',
    'numpy'
]

for package in REQUIRED_PACKAGES:
    try:
        dist = pkg_resources.get_distribution(package)
        print('{} ({}) is installed'.format(dist.key, dist.version))
    except pkg_resources.DistributionNotFound:
        print('{} is NOT installed'.format(package))
        install(package)
        print('{} was successfully installed.'.format(package))

biopython (1.83) is installed
pandas (2.1.4) is installed
numpy (1.26.1) is installed


In [1]:
# Import necessary libraries. 
import pandas as pd
import numpy as np

In [6]:
dataset = pd.read_csv('species.csv')

dataset.head()

,taxo_id,specie,blast_name,genbank_common_name,accession_number,mtDNA,Unnamed: 6
0,8945.0,Eudynamys scolopaceus,birds,Asian koel,NC_060520,https://www.ncbi.nlm.nih.gov/nucleotide/NC_060...,NaN
1,7460.0,Apis mellifera,Bees,honey bee,NC_051932,https://www.ncbi.nlm.nih.gov/nucleotide/NC_051...,NaN
2,36300.0,Pelecanus crispus,birds,Dalmatian pelican,OR620163,https://www.ncbi.nlm.nih.gov/nuccore/OR620163.1,NaN
3,10116.0,Rattus norvegicus,Rodents,Norway rat,NC_001665,https://www.ncbi.nlm.nih.gov/nuccore/NC_001665,NaN
4,9031.0,Gallus gallus,birds,Gallus gallus,NC_053523,https://www.ncbi.nlm.nih.gov/nuccore/NC_053523.1,NaN


In [4]:
import pandas as pd
import requests

# TODO: Add 10 more species to your dataset
# for species in additional_species:
    # Fetch mtDNA and add to the dataset

# TODO: Save the expanded dataset
# dataset.to_csv('./dataset/expanded_dataset.csv', index=False)


In [7]:
import pandas as pd
import requests
from Bio import Entrez
import os
from io import BytesIO
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Set your email and API key for NCBI
entrez_email = os.getenv('moien0813@gmail.com')  
entrez_key = os.getenv('f25054f1f99bce38c9f20403bab0f825ef08')
Entrez.email = entrez_email 

def fetch_entrez_record(db, id, rettype, retmode):
    """Fetch record from NCBI Entrez with retries."""
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db={db}&id={id}&rettype={rettype}&retmode={retmode}"
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    try:
        response = session.get(url)
        response.raise_for_status()
        if db == "nucleotide":
            return response.text
        return response.content
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error: {err}")
    except requests.exceptions.ConnectionError as err:
        print(f"Connection error: {err}")
    return None

def verify_taxonomy_id(taxo_id, species_name):
    """Verify taxonomy ID against species name."""
    xml_data = fetch_entrez_record("taxonomy", taxo_id, "xml", "xml")
    if xml_data:
        # Convert bytes data to a binary file-like object
        xml_data_io = BytesIO(xml_data)
        records = Entrez.read(xml_data_io)
        return records[0]['ScientificName'].lower() == species_name.lower()
    return False

def verify_mitochondrial_dna(accession_number):
    gb_data = fetch_entrez_record("nucleotide", accession_number, "gb", "text")
    return "mitochondrion" in gb_data.lower() if gb_data else False

def extract_accession_from_link(link):
    return link.split('/')[-1].split('.')[0]

def check_dataset_consistency(file_path):
    species_df = pd.read_csv(file_path)

    for index, row in species_df.iterrows():
        taxonomy_id = str(row['taxo_id'])
        species_name = row['specie']
        accession_number = row['accession_number']
        mtDNA_link = row['mtDNA']
        extracted_accession = extract_accession_from_link(mtDNA_link)

        taxonomy_check = verify_taxonomy_id(taxonomy_id, species_name)
        accession_match = (accession_number == extracted_accession)
        mitochondrial_check = verify_mitochondrial_dna(accession_number)

        print(f"Row {index}: Taxonomy Check: {taxonomy_check}, Accession Match: {accession_match}, Mitochondrial Check: {mitochondrial_check}")
        

In [12]:
check_dataset_consistency('species.csv')

Row 0: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 1: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 2: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 3: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 4: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 5: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 6: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 7: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 8: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 9: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 10: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 11: Taxonomy Check: True, Accession Match: True, Mitochondrial Check: True
Row 12: Taxonomy Check: True, Accession Match: True, Mitochond

In [ ]:
from Bio import Entrez, SeqIO
import csv

# TODO: Write a function to download mtDNA sequences
# def download_mtDNA(url, label):
    # Code to download and label the sequence
def download_fasta(accession, output_file):
    Entrez.email = 'moien0813@gmail.com'

    handle = Entrez.efetch(db='nucleotide', id=accession, rettype='fasta', retmode='text')
    
    with open(output_file, 'w') as file:
        file.write(handle.read())

    handle.close()

# TODO: Loop through the dataset and download each mtDNA sequence
# for index, row in dataset.iterrows():
    # Call the download function for each species
csv_file_path = 'species.csv'
with open(csv_file_path, 'r') as file:
    reader = csv.reader(file)
    header = next(reader)
    column_index = header.index('accession_number')
    accessions = [row[4] for row in reader]


# Downloading FASTA files for each accession
for accession in accessions:
    output_file = f'{accession}.fasta'
    download_fasta(accession, output_file)
    print(f"Downloaded FASTA file for accession {accession}")

In [ ]:
#concatenating downloaded fasta files
input_files = []
for i in accessions:
    input_files.append(i+".fasta")


output_file = 'concatenated.fasta'

with open(output_file, 'w') as out_handle:
    for fasta_file in input_files:
        with open(fasta_file, 'r') as in_handle:
            records = SeqIO.parse(in_handle, 'fasta')
        
            SeqIO.write(records, out_handle, 'fasta')

In [ ]:
# TODO: Install and import the alignment tool
# !pip install mafft
# from Bio.Align.Applications import MafftCommandline

# TODO: Perform the sequence alignment
# def perform_alignment(input_file, output_file):
    # Code to align sequences using the chosen tool

# TODO: Align your downloaded sequences
# perform_alignment('path_to_downloaded_sequences.fasta', 'aligned_sequences.fasta')

from Bio.Align.Applications import MuscleCommandline
from Bio import AlignIO


input_file = 'concatenated.fasta'
output_file = 'output.fasta'

muscle_cline = MuscleCommandline(input=input_file, out=output_file)

muscle_cline()

alignment = AlignIO.read(output_file, 'fasta')
print(alignment)

### Conclusion and Reflective Insights

As we conclude our exploration of phylogenetic tree construction and analysis, let's reflect on the insights learned from this task and consider questions that emerge from our findings.

#### Interpretation of Results:

- Reflect on the phylogenetic trees produced by each method (Bayesian inference, maximum likelihood, and neighbor-joining). Consider how the differences in tree topology might offer varied perspectives on the evolutionary relationships among the species.

#### Questions to Ponder:

1. **Species Divergence**: Based on the trees, which species appear to have the most ancient divergence? How might this information contribute to our understanding of their evolutionary history?
   
2. **Common Ancestors**: Are there any unexpected pairings or groupings of species that suggest a closer evolutionary relationship than previously thought? How could this reshape our understanding of these species' evolutionary paths?

3. **Methodology Insights**: Considering the discrepancies between the trees generated by different methods, what might this tell us about the limitations and strengths of each phylogenetic analysis method?

4. **Conservation Implications**: Considering the evolutionary relationships revealed in your phylogenetic analysis, what insights can be gained for conservation strategies? Specifically, how could understanding the close evolutionary ties between species, which might be facing distinct environmental challenges, guide targeted conservation efforts?

<blockquote style="font-family:Arial; color:red; font-size:16px; border-left:0px solid red; padding: 10px;">
    <strong>Don't forget to answer these questions!</strong>
</blockquote>
